In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

## Armado del dataset

#### Votos validos

In [ ]:
df = pd.read_csv('presidente_afirmativos.csv')

In [ ]:
df.shape

#### Mesas

In [ ]:
total_data_mesas = pd.read_csv('total_data_mesas.csv')

In [ ]:
total_data_mesas.shape

## Distribucion de mesas por % sacado

In [ ]:
for a, a_df in df.groupby('agrupacion_nombre'):
    fig = plt.figure(figsize=(10, 5))
    plt.title(a)
    a_df['percentage'].hist(bins=100)

## Mesas con % de votos de un partido

Defino el porcentaje votos de un partido para comparar

In [ ]:
set_percentage = 0.8

In [ ]:
mesas_percentage = total_data_mesas[total_data_mesas['percentage'] >= set_percentage]

#### Cantidad de mesas por agrupacion

In [ ]:
mesas_percentage['agrupacion_ganadora'].value_counts()

#### Cantidad de mesas por agrupacion y location

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_ganadora', 'distrito_nombre']).count().sort_values(
    ['agrupacion_ganadora', 'votos_cantidad'], ascending=False)

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'distrito_nombre', 'seccion_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_ganadora', 'distrito_nombre', 'seccion_nombre']).count().sort_values(
    ['agrupacion_ganadora', 'votos_cantidad'], ascending=False)

#### Cantidad de votos por agrupacion

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'votos_cantidad']].groupby(
    'agrupacion_ganadora').sum().sort_values('votos_cantidad', ascending=False)

#### Cantidad de mesas por agrupacion y distrito

In [ ]:
mesas_percentage[['agrupacion_ganadora', 'distrito_nombre', 'votos_cantidad']].groupby(
    ['agrupacion_ganadora', 'distrito_nombre']).sum().sort_values(
    ['agrupacion_ganadora', 'votos_cantidad'], ascending=False)

## Porcentaje de victoria

#### Total

In [ ]:
percentage_dfs = []

In [ ]:
for p, p_df in total_data_mesas.groupby('agrupacion_ganadora'):
    temp_df = p_df['percentage'].describe().to_frame()
    temp_df.columns = [p]
    percentage_dfs.append(temp_df.T)

In [ ]:
pd.concat(percentage_dfs)

## Votos no validos

#### % de mesas sin invalidos por agrupacion ganadora

In [ ]:
zero_invalidos = total_data_mesas[total_data_mesas['total_invalidos'] == 0].copy()

In [ ]:
agrupacion_grp_sin = zero_invalidos['agrupacion_ganadora'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_sin.columns = ['agrupacion_ganadora', 'sin_invalidos']

In [ ]:
agrupacion_grp_total = total_data_mesas['agrupacion_ganadora'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_total.columns = ['agrupacion_ganadora', 'total']

In [ ]:
agrupaciones_ganadoras_invalidos = pd.merge(agrupacion_grp_sin, agrupacion_grp_total)

In [ ]:
agrupaciones_ganadoras_invalidos['percentage'] = agrupaciones_ganadoras_invalidos[
    'sin_invalidos'] / agrupaciones_ganadoras_invalidos['total']

In [ ]:
agrupaciones_ganadoras_invalidos.sort_values('percentage', ascending=False)

#### Mesas ganadores con mas invalidos por agrupacion

In [ ]:
dfs = []

In [ ]:
for p, p_df in total_data_mesas.groupby('agrupacion_ganadora'):
    temp_df = p_df['invalidos_percentage'].describe().to_frame()
    temp_df.columns = [p]
    dfs.append(temp_df.T)

In [ ]:
inv_dist_df = pd.concat(dfs)

In [ ]:
inv_dist_df

In [ ]:
for v in ['mean', '25%', '50%', '75%', 'max']:
    fig = plt.figure(figsize=(15, 5))
    plt.title(v)
    plot_df = inv_dist_df.sort_values(v)
    x_values = plot_df.index
    plt.bar(x_values, plot_df[v])

## Votos en blanco

#### Votos en blanco por mesa por agrupacion ganadora

In [ ]:
blancos_por_mesa_ganadora = total_data_mesas[
    ['agrupacion_ganadora', 'EN BLANCO']].groupby('agrupacion_ganadora').agg(['count', 'sum'])

In [ ]:
blancos_por_mesa_ganadora

In [ ]:
blancos_por_mesa_ganadora['blancos_por_mesa'] = blancos_por_mesa_ganadora['EN BLANCO'][
    'sum'] / blancos_por_mesa_ganadora['EN BLANCO']['count']

In [ ]:
blancos_por_mesa_ganadora.sort_values('blancos_por_mesa', ascending=False)

#### Mesas con y sin votos en blanco

In [ ]:
con_votos_en_blanco = total_data_mesas[total_data_mesas['EN BLANCO'] > 0].copy()

In [ ]:
con_votos_en_blanco['agrupacion_ganadora'].value_counts()

In [ ]:
agrupacion_grp_con_votos_en_blanco = con_votos_en_blanco[
    'agrupacion_ganadora'].value_counts().to_frame().reset_index()

In [ ]:
agrupacion_grp_con_votos_en_blanco.columns = ['agrupacion_ganadora', 'con_blancos']

In [ ]:
agrupaciones_ganadoras_blancos = pd.merge(agrupacion_grp_con_votos_en_blanco, agrupacion_grp_total)

In [ ]:
agrupaciones_ganadoras_blancos['percentage'] = agrupaciones_ganadoras_blancos[
    'con_blancos'] / agrupaciones_ganadoras_invalidos['total']

In [ ]:
agrupaciones_ganadoras_blancos.sort_values('percentage', ascending=False)

## Mesas con un minimo de votos

Defino la cantidad minima de votos para el analisis

In [ ]:
min_votos = 10

In [ ]:
mesas_min_votos = df[df['votos_cantidad'] <= min_votos]

#### Total por agrupacion

In [ ]:
mesas_min_votos['agrupacion_nombre'].value_counts()

#### Por agrupacion y distrito

In [ ]:
agrupaciones = ['JUNTOS POR EL CAMBIO', 'LA LIBERTAD AVANZA', 'UNION POR LA PATRIA']

In [ ]:
mesas_min_votos[mesas_min_votos['agrupacion_nombre'].isin(agrupaciones)][
    ['distrito_nombre', 'agrupacion_nombre', 'mesa_id']].groupby(
    ['agrupacion_nombre', 'distrito_nombre']).count()

#### En un partido especifico

In [ ]:
# partido = 'LA LIBERTAD AVANZA'
partido = 'UNION POR LA PATRIA'

In [ ]:
partido_mesas_min_votos = mesas_min_votos[mesas_min_votos['agrupacion_nombre'] == partido]

In [ ]:
partido_a_comparar = 'LA LIBERTAD AVANZA'

In [ ]:
comparacion_df = df[df['agrupacion_nombre'] == partido_a_comparar]

In [ ]:
comparacion_partido = pd.merge(comparacion_df, partido_mesas_min_votos[
    ['distrito_nombre', 'seccionprovincial_nombre', 'seccion_nombre',
       'circuito_nombre', 'mesa_id']])

#### Distribucion por location

In [ ]:
comparacion_partido[['distrito_nombre', 'seccion_nombre', 'mesa_id']].groupby(
    ['distrito_nombre', 'seccion_nombre']).count().sort_values(
    ['mesa_id'], ascending=False).head()

#### Cantidad de votos en otro partido

In [ ]:
comparacion_partido['votos_cantidad'].sum()

In [ ]:
comparacion_partido[['distrito_nombre', 'seccion_nombre', 'votos_cantidad']].groupby(
    ['distrito_nombre', 'seccion_nombre']).sum().sort_values(
    ['votos_cantidad'], ascending=False).head(20)